In [1]:
import json
from matplotlib import pyplot as plt
from collections import defaultdict
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
import sklearn
import numpy as np
import pandas as pd
import random
import gzip
import math
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
import csv
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)


def SSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) 



In [ ]:
df = pd.read_json('Amazon_Fashion.jsonl.gz', compression='gzip', lines = True)

df = df.sample(df.shape[0], replace= False) #shuffle data through sampling without replacement

allRatings = df.get(["user_id", "asin", "rating"])
allRatings_filtered = allRatings[allRatings['user_id'].map(allRatings['user_id'].value_counts()) > 0].take(np.arange(50000))



allRatings_numpy = allRatings.to_numpy()

test_size = 5000
ratingsTrain = allRatings_filtered.to_numpy()
ratingsValid = allRatings_numpy[-test_size:]

ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))



N = ratingsTrain.shape[0]
nUsers = len(ratingsPerUser)
nItems = len(ratingsPerItem)
users = list(ratingsPerUser.keys())
items = list(ratingsPerItem.keys())


alpha_base = sum([r for _, _, r in ratingsTrain]) / ratingsTrain.shape[0]
alpha = alpha_base

userBiases = defaultdict(float)
itemBiases = defaultdict(float)


def prediction(user, item):
    return alpha + userBiases.get(user, 0) + itemBiases.get(item, 0)


def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d[0], d[1]) for d in ratingsTrain]
    cost = SSE(predictions, labels)
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost


def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(ratingsTrain)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for u,i,r in ratingsTrain:
        pred = prediction(u, i)
        diff = pred - r
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return np.array(dtheta)

print("hi")

labels = [r for _, _, r in ratingsTrain]
lamb = 0.00001795
for i in np.power(10., np.linspace(-5, -2, 5)):
    alpha = alpha_base
    userBiases = defaultdict(float)
    itemBiases = defaultdict(float)
    lamb = i
    result = scipy.optimize.fmin_l_bfgs_b(cost, [alpha] + [0.0] * (nUsers + nItems), derivative, args=(labels, lamb))
    unpack(result[0])

    validMSE = sum((prediction(u,b) - r) ** 2 for u, b, r in ratingsValid) /len(ratingsValid)
    print(lamb)
    print(validMSE)

hi
1e-05
2.031011005290374
5.623413251903491e-05
2.0365455804545265
0.00031622776601683794
1.9833165326701572
0.0017782794100389228
1.9895842557669083
0.01
1.991673487316493


### Results of Analysis

#### Parameters and Setup:
- **Filter**: Only users with at least k reviews (as well for training as for testing)
- **Dataset Size**: 50,000 reviews
- **Training Split**: 90%

#### Lambda Values and Results:

| **Lambda**           | **Result**           |
|-----------------------|----------------------|
| 1e-10                | 1.8639491452201058  |
| 1e-09                | 1.8900200977636963  |
| 1e-08                | 1.8607871149606283  |
| 1e-07                | 1.8761312052228225  |
| 1e-06                | 1.8313797079517349  |
| 1e-05                | 1.7754904375261358  |
| 0.0001               | 1.7256298986720695  |
| 0.001                | 1.7244053347977821  |
| 0.01                 | 1.7264580268739516  |
| 0.0005623413251903491 | 1.72385253847142    |
| 0.0031622776601683794 | 1.7256362562196657 |
| 0.01778279410038923  | 1.7264580313614348 |
| 0.1                  | 1.726710757329759   |

##### Only Users with at Least 3 Reviews:
| **Lambda**           | **Result**           |
|-----------------------|----------------------|
| 1e-05                | 1.8536334629174254  |
| 5.623413251903491e-05 | 1.814955153595779   |
| 0.00031622776601683794 | 1.811577437007883  |
| 0.0017782794100389228 | 1.814452901682741  |
| 0.01                 | 1.8162453086318369  |


##### Only Users with at Least 4 Reviews:
| **Lambda**           | **Result**           |
|-----------------------|----------------------|
| 1e-05                | 1.6352282627340333  |
| 5.623413251903491e-05 | 1.6000952481461368  |
| 0.00031622776601683794 | 1.5973304474433379 |
| 0.0017782794100389228 | 1.600359785941214  |
| 0.01                 | 1.6012302843086172  |

### Results of Analysis 2

#### Dataset Details:
- **Training Data**: 50,000 samples
- **Testing Data**: 5 samples

#### Experiments and Results:

##### **Train on Users with > 2 Reviews, Test Without Filter**
| **Lambda**           | **Result**           |
|-----------------------|----------------------|
| 1e-05                | 2.222831108785055   |
| 5.623413251903491e-05 | 2.2016408675758536  |
| 0.00031622776601683794 | 2.212174761502155  |
| 0.0017782794100389228 | 2.2222240581797887  |
| 0.01                 | 2.225215715699154   |

##### **Train on Users with > 1 Review, Test Without Filter**
| **Lambda**           | **Result**           |
|-----------------------|----------------------|
| 1e-05                | 2.015386428126356   |
| 5.623413251903491e-05 | 1.990013039961254   |
| 0.00031622776601683794 | 1.993702501227149  |
| 0.0017782794100389228 | 2.000484144770286  |
| 0.01                 | 2.0028333420670346  |

##### **Without Any Filter**
| **Lambda**           | **Result**           |
|-----------------------|----------------------|
| 1e-05                | 2.031011005290374   |
| 5.623413251903491e-05 | 2.0365455804545265  |
| 0.00031622776601683794 | 1.9833165326701572 |
| 0.0017782794100389228 | 1.9895842557669083 |
| 0.01                 | 1.991673487316493   |
